In [1]:
import pandas as pd
import numpy as np

In [2]:
train=pd.read_csv('titanic/train.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'titanic/train.csv'

In [ ]:
test=pd.read_csv('titanic/test.csv')

In [4]:
print(train.shape,test.shape)

(891, 12) (418, 11)


In [5]:
train_test_dataset=[train,test]

In [6]:
def cabin_mapping(cabin):
    if cabin == 'A':
        return 0
    elif cabin == 'B':
        return 0.4
    elif cabin == 'C':
        return 0.8
    elif cabin == 'D':
        return 1.2
    elif cabin == 'E':
        return 1.6
    elif cabin == 'F':
        return 2.0
    elif cabin == 'G':
        return 2.4
    elif cabin == 'T':
        return 2.8
    else:
        return

In [7]:
for data in train_test_dataset:
    data['Title']=data['Name'].str.extract('([a-zA-Z]+)\.')
    
    data['Title']=data['Title'].str.lower()
  
    data['Title']=np.where(data['Title']=='mr',0,
    np.where(data['Title']=='miss',1,
    np.where(data['Title']=='mrs',2,3)))
    
    data['Age']=data['Age'].fillna(data.groupby('Title')['Age'].transform('median'))
    
    data['Age']=np.where(data['Age']<=16,0,
    np.where(data['Age']<=26,1,
    np.where(data['Age']<=36,2,
    np.where(data['Age']<=62,3,4))))       
    data['Embarked']=np.where(data['Embarked']=='S',0,
    np.where(data['Embarked']=='C',1,2,))
    
    
    data['Sex']=data['Sex'].map({'male':0,'female':1})
    
    data['Fare'].fillna(data.groupby('Pclass')['Fare'].transform('median'))
    
    family_mapping={}
    for i,j in enumerate(range(1,12)):
        family_mapping[j]=round(i*0.4,1)
    data['FamilySize']=data['SibSp']+data['Parch']+1
    data['FamilySize']=data['FamilySize'].map(family_mapping)
    
    data['Cabin']= data['Cabin'].str[0]
    data['Cabin'] = data['Cabin'].apply(cabin_mapping)

    data['Cabin'].fillna(data.groupby('Pclass')['Cabin'].transform('median'),inplace=True)
    
    data['Fare']=np.where(data['Fare']<17,0,
    np.where(data['Fare']<=30,1,
    np.where(data['Fare']<100,2,3)))
         

In [8]:
features_drop=['PassengerId','Name','SibSp','Parch','Ticket']

In [9]:
train=train.drop(features_drop,axis=1)

In [10]:
test=test.drop(features_drop,axis=1)

In [11]:
train.head(1)

,Survived,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,FamilySize
0,0,3,0,1,0,2.0,0,0,0.4


In [12]:
test.head(1)

,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,FamilySize
0,3,0,2,0,2.0,2,0,0.0


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [72]:
train.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age           0
Fare          0
Cabin         0
Embarked      0
Title         0
FamilySize    0
dtype: int64

In [71]:
test.isnull().sum()

Pclass        0
Sex           0
Age           0
Fare          0
Cabin         0
Embarked      0
Title         0
FamilySize    0
dtype: int64

In [16]:
inputs=train.drop('Survived',axis=1)
#train.drop(columns='Survived')

In [17]:
outputs=train['Survived']

In [18]:
### 891 갯수가 맞아야 한다!! 확인!
print(inputs.shape,outputs.shape)

(891, 8) (891,)


In [19]:
## input=문제 output=  ####순서 정확하게 확인하기 x(input),y(output)
train_input, test_input, train_output, test_output =\
train_test_split(inputs,outputs, test_size=0.2, stratify=outputs, random_state=10)#stratify=outputs 비율을 유지하면서 점수를 뽑아라 #####random_state=10랜덤값 고정하기

In [20]:
print(train_input.shape, train_output.shape)
print(test_input.shape, test_output.shape)

(712, 8) (712,)
(179, 8) (179,)


In [21]:
test_input.iloc[0]

Pclass        3.0
Sex           1.0
Age           0.0
Fare          1.0
Cabin         2.0
Embarked      0.0
Title         1.0
FamilySize    1.6
Name: 24, dtype: float64

In [22]:
train_output.value_counts()

0    439
1    273
Name: Survived, dtype: int64

In [23]:
test_output.value_counts()

0    110
1     69
Name: Survived, dtype: int64

In [24]:
model1 = LogisticRegression()
model2 = DecisionTreeClassifier()
model3 = RandomForestClassifier()
model4 = GradientBoostingClassifier()
model5 = AdaBoostClassifier()

In [25]:
model1.fit(train_input, train_output)

LogisticRegression()

In [26]:
model1.score(train_input,train_output)*100

82.58426966292134

In [27]:
model1.score(test_input,test_output)*100

79.88826815642457

In [28]:
test_pre=model1.predict(test_input)

In [29]:
test_pre

array([0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0], dtype=int64)

In [30]:
test_pre[0]

0

In [75]:
test_output.iloc()

In [31]:
test_output.iloc[0]

0

In [32]:
accuracy_score(test_pre,test_output)*100

79.88826815642457

In [33]:
model2.fit(train_input, train_output)

DecisionTreeClassifier()

In [34]:
model2.score(train_input,train_output)*100

90.30898876404494

In [35]:
model2.score(test_input,test_output)*100

79.3296089385475

In [36]:
model2.predict(test_input)

array([0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int64)

In [37]:
model3.fit(train_input,train_output)

RandomForestClassifier()

In [38]:
model3.score(train_input,train_output)*100

90.30898876404494

In [39]:
model3.score(test_input,test_output)*100

81.56424581005587

In [40]:
model4.fit(train_input,train_output)

GradientBoostingClassifier()

In [41]:
model4.score(train_input,train_output)*100

86.51685393258427

In [42]:
model4.score(test_input,test_output)*100

83.24022346368714

In [43]:
model5.fit(train_input,train_output)

AdaBoostClassifier()

In [44]:
model5.score(train_input,train_output)*100

81.32022471910112

In [45]:
model5.score(test_input,test_output)*100

83.24022346368714

In [46]:
model={'LogisticRegression' : LogisticRegression(),
       'DecisionTreeClassifier' : DecisionTreeClassifier(),
       'RandomForestClassifier' : RandomForestClassifier(),
       'GradientBoostingClassifier' : GradientBoostingClassifier(),
       'AdaBoostClassifier' : AdaBoostClassifier()}



In [47]:
for i in model:
    clf=model[i]
    clf.fit(train_input,train_output)
    train_score=clf.score(train_input,train_output)*100
    test_score=clf.score(test_input,test_output)*100
    print("{0} train:{1}/ test:{2}" .format(i,train_score, test_score))

LogisticRegression train:82.58426966292134/ test:79.88826815642457
DecisionTreeClassifier train:90.30898876404494/ test:79.3296089385475
RandomForestClassifier train:90.30898876404494/ test:82.68156424581005
GradientBoostingClassifier train:86.51685393258427/ test:83.24022346368714
AdaBoostClassifier train:81.32022471910112/ test:83.24022346368714


In [48]:
model_1={}
train_1={}
test_1={}
for i in model:
    clf=model[i]
    clf.fit(train_input,train_output)
    train_score=clf.score(train_input,train_output)*100
    test_score=clf.score(test_input,test_output)*100
    print("{0} train:{1}/ test:{2}" .format(i,train_score, test_score))

LogisticRegression train:82.58426966292134/ test:79.88826815642457
DecisionTreeClassifier train:90.30898876404494/ test:79.3296089385475
RandomForestClassifier train:90.30898876404494/ test:82.68156424581005
GradientBoostingClassifier train:86.51685393258427/ test:83.24022346368714
AdaBoostClassifier train:81.32022471910112/ test:83.24022346368714


In [50]:
data=list(zip(model_1,train_1,test_1))

In [89]:
data

[]

In [52]:
model= GradientBoostingClassifier()

In [53]:
model.fit(train_input,train_output)

GradientBoostingClassifier()

In [54]:
import pickle

In [55]:
with open('gbc.pkl','wb') as f:
    pickle.dump(model,f)

In [56]:
with open('gbc.pkl','rb') as f:
    load_model=pickle.load(f)

In [57]:
load_model

GradientBoostingClassifier()

In [58]:
test_input.iloc[0].to_list()

[3.0, 1.0, 0.0, 1.0, 2.0, 0.0, 1.0, 1.6]

In [59]:
import numpy as np

In [60]:
np_data=np.asarray(test_input.iloc[0].to_list())

In [83]:
re_data=np_data.reshape(1,-1)

In [84]:
re_data

array([[3. , 1. , 0. , 1. , 2. , 0. , 1. , 1.6]])

In [85]:
load_model.predict(re_data)

C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


array([0], dtype=int64)

---

In [86]:
test

,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,FamilySize
0,3,0,2,0,2.0,2,0,0.0
1,3,1,3,0,2.0,0,2,0.4
2,2,0,3,0,2.0,2,0,0.0
3,3,0,2,0,2.0,0,0,0.0
4,3,1,1,0,2.0,0,2,0.8
...,...,...,...,...,...,...,...,...
413,3,0,2,0,2.0,0,0,0.0
414,1,1,3,3,0.8,1,3,0.0
415,3,0,3,0,2.0,0,0,0.0
416,3,0,2,0,2.0,0,0,0.0


In [87]:
from sklearn.ensemble import GradientBoostingClassifier

In [88]:
sub=pd.read_csv('titanic/submission.csv')

In [67]:
answer=sub['Survived']

In [68]:
load_model

GradientBoostingClassifier()

In [77]:
submission=load_model.predict(test)

In [78]:
accuracy_score(answer,submission)*100

85.4066985645933

In [79]:
sub2=pd.read_csv('titanic/submission2.csv')

In [80]:
sub2

,PassengerId,Survived
0,892,NaN
1,893,NaN
2,894,NaN
3,895,NaN
4,896,NaN
...,...,...
413,1305,NaN
414,1306,NaN
415,1307,NaN
416,1308,NaN


In [81]:
sub2['Survived']=submission

In [82]:
sub2.to_csv('result.csv', index=False)